# **Challenge LATAM**

En este documento se encuentra la descripción detallada de cada uno de los puntos del challenge. Como se pensaron las soluciones los tiempos y memoria utilizada.

# Memory-Optimized Version of q1_memory Function


## Overview
Esta función está diseñada para analizar datos de tweets y extraer información sobre las fechas principales con más tweets y los usuarios con más publicaciones para cada una de estas fechas. El código está optimizado para la eficiencia de la memoria.

## Dependencies
- Python 3.x
- pandas
- memory-profiler (for memory profiling)

## Function Signature
```python
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:


## Parameters
- `file_path` (str): El camino al archivo JSON que contiene los datos de los tweets.

## Return Value
- Lista de Tuplas: Cada tupla contiene una fecha (objeto datetime.date) y el nombre de usuario (str) con más publicaciones en esa fecha.

## Memory Optimization Techniques
1. **Carga directa del DataFrame:** El DataFrame se carga directamente desde el archivo JSON sin operaciones de agrupamiento innecesarias.
2. **Filtrado eficiente de fechas:** Las fechas con más tweets se identifican usando `value_counts().head(10)`, optimizando el uso de memoria.
3. **Operaciones vectorizadas:** Uso de operaciones vectorizadas como `isin` y `groupby` para una manipulación eficiente de datos.
4. **Creación mínima de DataFrames:** Los DataFrames se crean solo cuando es necesario para minimizar el consumo de memoria.


## Memory Profiling

In [8]:
from typing import List, Tuple
from datetime import datetime
from utils import load_json_as_df
import pandas as pd
import psutil

def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    # Obtener el uso de memoria antes de cargar el DataFrame
    memory_before = psutil.Process().memory_info().rss / 1024 ** 2  # en MB

    # Cargar el DataFrame directamente en lugar de cargarlo y luego agruparlo
    df = load_json_as_df(file_path, include_date=True)

    # Obtener el uso de memoria después de cargar el DataFrame
    memory_after_load = psutil.Process().memory_info().rss / 1024 ** 2  # en MB

    # Obtener las fechas con más tweets directamente sin agrupar todo el DataFrame
    top_dates = df['date'].value_counts().head(10).index

    # Filtrar el DataFrame solo para las fechas con más tweets
    top_df = df[df['date'].isin(top_dates)]

    # Obtener el uso de memoria después de filtrar el DataFrame
    memory_after_filter = psutil.Process().memory_info().rss / 1024 ** 2  # en MB

    # Obtener los usuarios con más publicaciones para cada fecha top
    top_users = top_df.groupby('date')['username'].agg(lambda x: x.value_counts().idxmax()).reset_index()

    # Convertir las fechas a objetos datetime.date
    top_users['date'] = pd.to_datetime(top_users['date']).dt.date

    # Obtener el uso de memoria después de crear el DataFrame final
    memory_after_final_df = psutil.Process().memory_info().rss / 1024 ** 2  # en MB

    # Obtener el resultado final como una lista de tuplas
    result = list(top_users.itertuples(index=False, name=None))

    # Imprimir información sobre el uso de memoria
    print(f"Uso de memoria antes de cargar el DataFrame: {memory_before} MB")
    print(f"Uso de memoria después de cargar el DataFrame: {memory_after_load} MB")
    print(f"Uso de memoria después de filtrar el DataFrame: {memory_after_filter} MB")
    print(f"Uso de memoria después de crear el DataFrame final: {memory_after_final_df} MB")

    return result

if __name__ == "__main__":
    result = q1_memory(r'C:\Users\wppaez\Desktop\challenge_LATAM\input\farmers-protest-tweets-2021-2-4.json')
    print(result)


Uso de memoria antes de cargar el DataFrame: 108.78125 MB
Uso de memoria después de cargar el DataFrame: 120.875 MB
Uso de memoria después de filtrar el DataFrame: 123.41796875 MB
Uso de memoria después de crear el DataFrame final: 122.73046875 MB
[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 19), 'Preetm91'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria')]


# Time-Optimized Version of q1_time Function

Esta función se utiliza para obtener las fechas con más tweets y los usuarios con más publicaciones para cada fecha a partir de un archivo JSON con datos de tweets.

## Parameters

- `file_path` (str): Ruta del archivo JSON con los datos de los tweets.

## Return value

Lista de tuplas que contiene la fecha y el usuario con más publicaciones para esa fecha.

## Usage

```python
result = q1_time('./input/farmers-protest-tweets-2021-2-4.json')
print(result)


## Time Optimization Techniques

1. Carga el DataFrame directamente en lugar de cargarlo y luego agruparlo.
2. Obtiene las fechas con más tweets directamente sin agrupar todo el DataFrame.
3. Filtra el DataFrame solo para las fechas con más tweets.
4. Obtiene los usuarios con más publicaciones para cada fecha top.
5. Convierte las fechas a objetos datetime.date.
6. Retorna el resultado final como una lista de tuplas.

In [4]:
import cProfile
from datetime import datetime
from utils import load_json_as_df
import pandas as pd

def q1_time(file_path: str):
    """
    Función para obtener las fechas con más tweets y los usuarios con más publicaciones
    para cada fecha a partir de un archivo JSON con datos de tweets.

    Args:
        file_path (str): Ruta del archivo JSON con los datos de los tweets.
    """

    # Cargar el DataFrame directamente en lugar de cargarlo y luego agruparlo
    df = load_json_as_df(file_path, include_date=True)

    # Obtener las fechas con más tweets directamente sin agrupar todo el DataFrame
    top_dates = df['date'].value_counts().nlargest(10).index

    # Filtrar el DataFrame solo para las fechas con más tweets
    top_df = df[df['date'].isin(top_dates)]

    # Obtener los usuarios con más publicaciones para cada fecha top
    top_users = top_df.groupby('date')['username'].apply(lambda x: x.mode().iloc[0]).reset_index()

    # Convertir las fechas a objetos datetime.date
    top_users['date'] = pd.to_datetime(top_users['date']).dt.date

    # Obtener el resultado final como una lista de tuplas
    result = list(top_users.itertuples(index=False, name=None))

    return result

# Ruta del archivo JSON con los datos de los tweets
file_path = r'C:\Users\wppaez\Desktop\challenge_LATAM\input\farmers-protest-tweets-2021-2-4.json'

# Crea un perfilador
profiler = cProfile.Profile()

# Inicia el perfilador
profiler.enable()

# Ejecuta la función que quieres medir
result = q1_time(file_path)

# Detén el perfilador
profiler.disable()

# Muestra las estadísticas del perfilador
profiler.print_stats(sort='time')


         4689027 function calls (4688182 primitive calls) in 14.651 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   117407    4.813    0.000    4.813    0.000 decoder.py:343(raw_decode)
        1    2.799    2.799   14.491   14.491 utils.py:6(load_json_as_df)
   117407    2.666    0.000    4.626    0.000 _strptime.py:309(_strptime)
   117407    0.636    0.000    5.262    0.000 _strptime.py:565(_strptime_datetime)
   352221    0.488    0.000    0.488    0.000 {method 'match' of 're.Pattern' objects}
   117407    0.412    0.000    5.582    0.000 decoder.py:332(decode)
   117407    0.372    0.000    0.372    0.000 {built-in method _locale.setlocale}
   117407    0.285    0.000    5.940    0.000 __init__.py:299(loads)
   117407    0.227    0.000    0.318    0.000 locale.py:396(normalize)
   117407    0.199    0.000    0.199    0.000 {method 'groupdict' of 're.Match' objects}
   117407    0.176    0.000    5.438    0.000 {bui

# Memory-Optimized Version of q2_memory Function

In [17]:
import pandas as pd
import emoji
import sys  # Módulo para obtener el tamaño de los objetos en memoria
from typing import List, Tuple
from utils import load_json_as_df

def extract_emojis(text: str):
    """Extrae emojis de un texto dado."""
    for c in text:
        if c in emoji.EMOJI_DATA:
            yield c

def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    """
    Calcula el uso de memoria al procesar un archivo JSON de tweets y contar los emojis más frecuentes.

    Parameters:
    file_path (str): Ruta del archivo JSON de tweets.

    Returns:
    List[Tuple[str, int]]: Lista de tuplas con los emojis más frecuentes y su conteo.
    """
    emojis_count = {}
    total_memory = 0  # Variable para llevar el total de memoria estimada
    batch_size = 1000  # Procesa el archivo por lotes de 1000 tweets
    for chunk in pd.read_json(file_path, lines=True, chunksize=batch_size):
        for text in chunk['content']:
            emojis = extract_emojis(text)
            for emoji in emojis:
                emojis_count[emoji] = emojis_count.get(emoji, 0) + 1
                total_memory += sys.getsizeof(emoji)  # Estimar el tamaño de cada emoji en memoria
    top_emojis = sorted(emojis_count.items(), key=lambda x: x[1], reverse=True)[:10]
    return top_emojis, total_memory

if __name__ == "__main__":
    file_path =  r'C:\Users\wppaez\Desktop\challenge_LATAM\input\farmers-protest-tweets-2021-2-4.json'
    result, estimated_memory = q2_memory(file_path)
    print(f"Uso de memoria estimado: {estimated_memory} bytes")
    print(result)


Uso de memoria estimado: 3625296 bytes
[('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🏻', 2080), ('❤', 1779), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]


## Memory Optimization Techniques emojis tweet

El código para procesar emojis en tweets está diseñado de manera eficiente para minimizar el uso de memoria. A continuación, se detallan las técnicas utilizadas para ahorrar memoria:

### 1. Procesamiento por Lotes (Batch Processing)

El código lee el archivo JSON de tweets por lotes utilizando el parámetro `chunksize` en la función `pd.read_json()`. Esto permite procesar los tweets en bloques pequeños en lugar de cargar todo el archivo en memoria de una vez, lo que reduce la carga en la memoria.

### 2. Extracción de Emojis en Tiempo Real

En lugar de almacenar todos los emojis encontrados en una estructura de datos en memoria, el código utiliza un generador para extraer emojis en tiempo real mientras se recorre el texto de cada tweet. Esto evita la necesidad de almacenar todos los emojis en memoria antes de procesarlos.

### 3. Uso Eficiente de Estructuras de Datos

Se utiliza un diccionario `emojis_count` para mantener un conteo de la frecuencia de cada emoji. En lugar de almacenar cada ocurrencia de un emoji como un objeto separado, el código actualiza directamente el conteo en el diccionario, lo que ahorra memoria al evitar la duplicación de datos.

### 4. Limitación de Resultados Finales

El código utiliza la función `sorted()` para obtener los 10 emojis más frecuentes y sus conteos. Esto limita la cantidad de datos almacenados en la memoria final, asegurando que solo se retengan los resultados más relevantes.

### 5. Estimación de Uso de Memoria

Para proporcionar información sobre el uso de memoria, se utiliza la función `sys.getsizeof()` para estimar el tamaño en bytes de cada emoji mientras se procesa. Esta estimación se suma para obtener una medida aproximada del uso de memoria total durante la ejecución del código.

En conjunto, estas técnicas garantizan un uso eficiente de la memoria al procesar emojis en tweets, minimizando la carga en la memoria y optimizando el rendimiento del código.


# Time-Optimized Version of q2_time Function

In [22]:
import cProfile
import pandas as pd
import emoji
from typing import List, Tuple
from utils import load_json_as_df

def extract_emojis(text: str) -> List[str]:
    return [c for c in text if c in emoji.EMOJI_DATA]

def q2_time(file_path: str) -> List[Tuple[str, int]]:
    df = load_json_as_df(file_path, include_content=True)
    
    # Crear un conjunto de emojis únicos
    unique_emojis = set(emoji.EMOJI_DATA.keys())
    
    # Contar emojis únicos en cada texto
    emojis_count = {}
    for text in df['content']:
        extracted_emojis = set(extract_emojis(text))  # Convertir a conjunto para emojis únicos
        emojis_in_text = extracted_emojis.intersection(unique_emojis)  # Filtrar emojis válidos
        for current_emoji in emojis_in_text:
            emojis_count[current_emoji] = emojis_count.get(current_emoji, 0) + 1
    
    # Obtener los 10 emojis más frecuentes
    top_emojis = sorted(emojis_count.items(), key=lambda x: x[1], reverse=True)[:10]
    return top_emojis

if __name__ == "__main__":
    cProfile.run('q2_time(r"C:\\Users\\wppaez\\Desktop\\challenge_LATAM\\input\\farmers-protest-tweets-2021-2-4.json")')


         1776130 function calls (1776124 primitive calls) in 9.232 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.187    0.187    9.213    9.213 3098816803.py:10(q2_time)
      641    0.000    0.000    0.000    0.000 3098816803.py:25(<lambda>)
   117407    0.067    0.000    1.790    0.000 3098816803.py:7(extract_emojis)
   117407    1.723    0.000    1.723    0.000 3098816803.py:8(<listcomp>)
        1    0.018    0.018    9.232    9.232 <string>:1(<module>)
   117407    0.215    0.000    4.831    0.000 __init__.py:299(loads)
        3    0.000    0.000    0.000    0.000 __init__.py:34(using_copy_on_write)
        1    0.000    0.000    0.000    0.000 __init__.py:42(warn_copy_on_write)
        5    0.000    0.000    0.000    0.000 __init__.py:55(using_pyarrow_string_dtype)
        1    0.000    0.000    0.000    0.000 _asarray.py:108(<setcomp>)
        1    0.000    0.000    0.000    0.000 _asarray.py:27(req

## Optimizations and Advantages:

En el código anterior, se implementaron varias optimizaciones para mejorar el rendimiento y la eficiencia:

Utilización de Conjuntos para Emojis Únicos: Al utilizar conjuntos (unique_emojis y emojis_in_text), se garantiza que solo se consideren emojis únicos, reduciendo las iteraciones y comparaciones redundantes.

Diccionario para Contar Emojis: El uso de un diccionario (emojis_count) para contar la aparición de cada emoji permite operaciones de búsqueda y actualización eficientes, evitando iteraciones innecesarias en todo el conjunto de datos.

Intersección para Emojis Válidos: La intersección de conjuntos (intersection(unique_emojis)) filtra los emojis que no están en el conjunto de emojis únicos, reduciendo el número de comparaciones y cálculos.

Ordenamiento de los Emojis Más Frecuentes: Al ordenar los emojis por conteo (sorted(emojis_count.items(), key=lambda x: x[1], reverse=True)[:10]), se asegura que solo se procesen y muestren los 10 emojis más frecuentes, ahorrando tiempo en el procesamiento de datos menos relevantes.

Extracción Eficiente de Emojis: La función extract_emojis extrae eficientemente emojis del texto utilizando comprensión de listas y la librería emoji, asegurando un proceso optimizado.

Estas optimizaciones conducen a tiempos de ejecución más rápidos al reducir cálculos innecesarios, aprovechar estructuras de datos como conjuntos y diccionarios para operaciones eficientes, y enfocarse en procesar solo los datos más relevantes. En comparación con enfoques sin estas optimizaciones, tu código se beneficia de una complejidad reducida y un rendimiento mejorado, especialmente al tratar con grandes conjuntos de datos o operaciones frecuentes en texto con emojis.

# Memory-Optimized Version of q3_memory Function

In [24]:
import pandas as pd
import emoji
from typing import List, Tuple
from utils import load_json_as_df
import os
import psutil

def extract_emojis(text: str) -> List[str]:
    return [c for c in text if c in emoji.EMOJI_DATA]

def q2_time(file_path: str) -> List[Tuple[str, int]]:
    df = load_json_as_df(file_path, include_content=True)
    
    # Crear un conjunto de emojis únicos
    unique_emojis = set(emoji.EMOJI_DATA.keys())
    
    # Contar emojis únicos en cada texto
    emojis_count = {}
    for text in df['content']:
        extracted_emojis = set(extract_emojis(text))  # Convertir a conjunto para emojis únicos
        emojis_in_text = extracted_emojis.intersection(unique_emojis)  # Filtrar emojis válidos
        for current_emoji in emojis_in_text:
            emojis_count[current_emoji] = emojis_count.get(current_emoji, 0) + 1
    
    # Obtener los 10 emojis más frecuentes
    top_emojis = sorted(emojis_count.items(), key=lambda x: x[1], reverse=True)[:10]
    return top_emojis

if __name__ == "__main__":
    file_path = r"C:\Users\wppaez\Desktop\challenge_LATAM\input\farmers-protest-tweets-2021-2-4.json"
    
    # Obtener el uso de memoria antes de ejecutar la función
    process = psutil.Process(os.getpid())
    memory_before = process.memory_info().rss
    
    # Ejecutar la función
    result = q2_time(file_path)
    
    # Obtener el uso de memoria después de ejecutar la función
    memory_after = process.memory_info().rss
    
    # Calcular la diferencia de uso de memoria
    memory_diff = memory_after - memory_before
    
    print(f"Uso de memoria antes de la ejecución: {memory_before} bytes")
    print(f"Uso de memoria después de la ejecución: {memory_after} bytes")
    print(f"Diferencia de uso de memoria: {memory_diff} bytes")


Uso de memoria antes de la ejecución: 110948352 bytes
Uso de memoria después de la ejecución: 127758336 bytes
Diferencia de uso de memoria: 16809984 bytes


## Efficient memory usage:

Los resultados muestran el uso de memoria antes y después de cargar y procesar el DataFrame, así como el uso de memoria estimado después de ejecutar la función q2_time. Aquí están las ventajas de cómo se optimizó el código en términos de uso de memoria:

Uso eficiente de la memoria al cargar el DataFrame:

Antes de cargar el DataFrame, el uso de memoria era de 108.78125 MB. Después de cargar el DataFrame, aumentó a 120.875 MB. Esto indica un uso eficiente de la memoria al cargar y manipular un archivo de datos grande (389 MB) utilizando pandas.
Uso controlado de la memoria durante el procesamiento de datos:

Después de filtrar el DataFrame y crear el DataFrame final, el uso de memoria fluctuó pero se mantuvo en un rango relativamente bajo, alrededor de 123.41796875 MB y 122.73046875 MB respectivamente. Esto indica que el código optimizado gestiona la memoria de manera efectiva durante el procesamiento de datos, evitando grandes aumentos en el uso de memoria.
Uso de memoria estimado después de la ejecución:

El uso de memoria estimado después de ejecutar la función q2_time fue de 3625296 bytes, que es aproximadamente 3.46 MB. Esto muestra una gestión eficiente de la memoria al realizar operaciones de conteo de emojis y selección de los 10 emojis más frecuentes.
En resumen, las ventajas de cómo se optimizó el código en términos de uso de memoria incluyen un manejo eficiente de la memoria al cargar y procesar el DataFrame, controlando el uso de memoria durante el procesamiento de datos y minimizando el uso de memoria al ejecutar operaciones específicas en los datos. Esto resulta en un uso más efectivo de los recursos de memoria, especialmente al trabajar con conjuntos de datos grandes como el mencionado de 389 MB.

# Time-Optimized Version of q3_time Function

In [28]:
import cProfile
from typing import List, Tuple
from utils import load_json_as_df
from collections import Counter

def q3_time(file_path: str) -> List[Tuple[str, int]]:
    # Load JSON data into DataFrame
    df = load_json_as_df(file_path, include_mentions=True)

    # Use Counter to count mentions directly in DataFrame
    mention_counter = Counter(df['username'])

    # Get top 10 users with mention counts
    top_10_users = mention_counter.most_common(10)
    return top_10_users

if __name__ == "__main__":
    # Use cProfile to profile the code and measure execution time
    cProfile.run('result=q2_time(r"C:\\Users\\wppaez\\Desktop\\challenge_LATAM\\input\\farmers-protest-tweets-2021-2-4.json")', sort='cumulative')


         1776149 function calls (1776143 primitive calls) in 8.499 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    8.499    8.499 {built-in method builtins.exec}
        1    0.026    0.026    8.499    8.499 <string>:1(<module>)
        1    0.179    0.179    8.474    8.474 2767133957.py:11(q2_time)
        1    1.851    1.851    6.483    6.483 utils.py:6(load_json_as_df)
   117407    0.197    0.000    4.430    0.000 __init__.py:299(loads)
   117407    0.302    0.000    4.174    0.000 decoder.py:332(decode)
   117407    3.649    0.000    3.649    0.000 decoder.py:343(raw_decode)
   117407    0.067    0.000    1.772    0.000 2767133957.py:8(extract_emojis)
   117407    1.706    0.000    1.706    0.000 2767133957.py:9(<listcomp>)
   234814    0.153    0.000    0.153    0.000 {method 'match' of 're.Pattern' objects}
    49772    0.067    0.000    0.143    0.000 codecs.py:319(decode)
    49772  


Basándome en los resultados del profiler y teniendo en cuenta que el archivo de entrada pesa 389 MB. El codigo se puede optimizar

Optimización de Carga de Datos:

Dado que la carga de datos desde un archivo JSON de 389 MB está consumiendo una cantidad significativa de tiempo (aproximadamente 7.768 segundos), considera utilizar técnicas de carga de datos más eficientes.
Una opción es cargar y procesar el archivo por lotes en lugar de cargarlo completo en memoria. Esto puede reducir la carga en el sistema y mejorar el tiempo.